In [ ]:
# DRF_filter_fut_inputs
# This script takes the 'raw' inputs generated by the DRF_simulation_stage
# and then filters them to give smoother control

# derivation is shown in Filter_derivation.jpg
from math import pi, degrees

if ii > 1:
    sign_delta_fut_filt = int(delta_fut * delta_fut_temp_prev >= 0)

    if sign_delta_fut_filt == -1 and abs(delta_fut - delta_fut_temp_prev) >= 10 * (pi / 180) * (1 / Sr):
        delta_fut_filt = 0
        print("steering filtered\n")
    else:
        delta_fut_filt = delta_fut

    v_fut_filt = v_fut
    delta_fut_temp_prev = delta_fut
else:
    delta_fut_filt = delta_fut
    v_fut_filt = v_fut
    delta_fut_temp_prev = delta_fut

# DRF_filter_fut_inputs
# This script takes the 'raw' inputs generated by the DRF_simulation_stage
# and then filters them to give smoother control

# derivation is shown in Filter_derivation.jpg
if ii > 1:
    delta_fut_filt = (dt / (Tfs + dt)) * delta_fut + (Tfs / (Tfs + dt)) * Delta[ii - 1]
    v_fut_filt = (dt / (Tfv + dt)) * v_fut + (Tfv / (Tfv + dt)) * V[ii - 1]
else:
    delta_fut_filt = delta_fut
    v_fut_filt = v_fut


import os
import sys

# Including the different directories used in this project
main_dir = os.getcwd()
os.chdir("..")

car_kin_dir = os.path.abspath("Car_kinematics")
gaus3d_dir = os.path.abspath("Gaussian_3d_torus")
head_cntrl_dir = os.path.abspath("Heading_controller")
drf_model_dir = os.path.abspath("DRF_model")
obstacles_dir = os.path.abspath("Obstacles")
plotting_dir = os.path.abspath("Plotting")
road_func_dir = os.path.abspath("Road_functions")
record_data_dir = os.path.abspath("recorded_data")
db_concept_dir = os.path.abspath("Debug_concept")
lat_dev_dir = os.path.abspath("Lateral_Deviation")

# Adding directories to the Python path
sys.path.extend([
    car_kin_dir, gaus3d_dir, head_cntrl_dir, drf_model_dir,
    obstacles_dir, plotting_dir, road_func_dir, record_data_dir,
    db_concept_dir, lat_dev_dir
])

os.chdir(main_dir)


import numpy as np
from scipy.optimize import minimize_scalar

# Define the cost function
def fcost(delta_optim, *args):
    # Assuming min_delta_cost_Ct() is already defined in Python
    return min_delta_cost_Ct(delta_optim, *args)

# Set the optimization bounds
bounds = (delta_fut_h - dstep_max, delta_fut_h + dstep_max)

# Define the arguments to be passed to the cost function
args = (phiv_a, tla, par1, cexp, v_inter_temp, L, xv, yv, res, kexp1, kexp2, mcexp, Env, ii, Ct, delta_fut_h, Pipe_sequence, Is, Obs_sequence, Os, dt, delta1, Car_Nrp_Idx)

# Perform the optimization
result = minimize_scalar(fcost, bounds=bounds, args=args)

# Get the minimum delta cost and the total cost
delta_cost_min = result.x
cost_min = result.fun + Ct


import numpy as np
from scipy.optimize import minimize_scalar

# Define the cost function
def fcost(delta_optim, *args):
    # Assuming min_delta_cost() is already defined in Python
    return min_delta_cost(delta_optim, *args)

# Set the optimization bounds
bounds = (delta_fut_h - dstep_max, delta_fut_h + dstep_max)

# Define the arguments to be passed to the cost function
args = (phiv_a, tla, par1, cexp, v_inter_temp, L, xv, yv, res, kexp1, kexp2, mcexp, Env, ii, Pipe_sequence, Is, Obs_sequence, Os, dt, delta1, Car_Nrp_Idx)

# Set the optimization options
options = {'xatol': 1e-6}

# Perform the optimization
result = minimize_scalar(fcost, bounds=bounds, args=args, options=options)

# Get the minimum delta cost and the total cost
delta_cost_min = result.x
cost_min = result.fun


import numpy as np

def min_delta_cost_Ct(delta_optim, phiv_a, tla, par1, cexp, v_inter_temp, L, xv, yv, res, kexp1, kexp2, mcexp, Env, ii, Ct, delta_fut_h, Pipe_sequence, Is, Obs_sequence, Os, dt, delta1, Car_Nrp_Idx):
    # Assuming all the required functions are already defined in Python
    delta = Gaussian_3d_torus_delta(delta_optim)
    phiv = Gaussian_3d_torus_phiv(phiv_a)
    dla = Gaussian_3d_torus_dla(tla, v_inter_temp)
    R = Gaussian_3d_torus_R(L, delta)
    xc, yc = Gaussian_3d_torus_xcyc(xv, yv, phiv, delta, R)
    X, Y, xbl, xbu, ybl, ybu = Gaussian_3d_torus_meshgrid(xv, yv, dla, res, Car_Nrp_Idx[ii])
    mexp1 = Gaussian_3d_torus_mexp(kexp1, mcexp, delta, v_inter_temp, delta1, dt)
    mexp2 = Gaussian_3d_torus_mexp(kexp2, mcexp, delta, v_inter_temp, delta1, dt)
    arc_len = Gaussian_3d_torus_arclen(X, Y, xv, yv, delta, xc, yc, R)
    a = Gaussian_3d_torus_a(arc_len, par1, dla)
    sigma1 = Gaussian_3d_torus_sigma(arc_len, mexp1, cexp)
    sigma2 = Gaussian_3d_torus_sigma(arc_len, mexp2, cexp)
    Z_cur = Gaussian_3d_torus_z(X, Y, xc, yc, R, a, sigma1, sigma2)
    Zroad_cur = Generate_circuit_road_solo_cost(X, Y, Env, Pipe_sequence, Is)
    Zobs_cur = Obstacle_cost(X, Y, Obs_sequence, Os, ii, dt, Car_Nrp_Idx)
    Zenv_cur = Zroad_cur + Zobs_cur
    Zcost_cur = Z_cur * Zenv_cur
    cost_cur = np.sum(Zcost_cur) * (res ** 2)
    cost_Ct = (cost_cur - 1.1 * Ct) ** 2

    return cost_Ct

import numpy as np

def min_delta_cost(delta_optim, phiv_a, tla, par1, cexp, v_inter_temp, L, xv, yv, res, kexp1, kexp2, mcexp, Env, ii, Pipe_sequence, Is, Obs_sequence, Os, dt, delta1, Car_Nrp_Idx):
    # Assuming all the required functions are already defined in Python
    delta = Gaussian_3d_torus_delta(delta_optim)
    phiv = Gaussian_3d_torus_phiv(phiv_a)
    dla = Gaussian_3d_torus_dla(tla, v_inter_temp)
    R = Gaussian_3d_torus_R(L, delta)
    xc, yc = Gaussian_3d_torus_xcyc(xv, yv, phiv, delta, R)
    X, Y, xbl, xbu, ybl, ybu = Gaussian_3d_torus_meshgrid(xv, yv, dla, res, Car_Nrp_Idx[ii])
    mexp1 = Gaussian_3d_torus_mexp(kexp1, mcexp, delta, v_inter_temp, delta1, dt)
    mexp2 = Gaussian_3d_torus_mexp(kexp2, mcexp, delta, v_inter_temp, delta1, dt)
    arc_len = Gaussian_3d_torus_arclen(X, Y, xv, yv, delta, xc, yc, R)
    a = Gaussian_3d_torus_a(arc_len, par1, dla)
    sigma1 = Gaussian_3d_torus_sigma(arc_len, mexp1, cexp)
    sigma2 = Gaussian_3d_torus_sigma(arc_len, mexp2, cexp)
    Z_cur = Gaussian_3d_torus_z(X, Y, xc, yc, R, a, sigma1, sigma2)
    Zroad_cur = Generate_circuit_road_solo_cost(X, Y, Env, Pipe_sequence, Is)
    Zobs_cur = Obstacle_cost(X, Y, Obs_sequence, Os, ii, dt, Car_Nrp_Idx)
    Zenv_cur = Zroad_cur + Zobs_cur
    Zcost_cur = Z_cur * Zenv_cur
    cost_cur = np.sum(Zcost_cur) * (res ** 2)

    return cost_cur


In [ ]:
import numpy as np

# === Simulation parameters ===
Sr = 54.9038         # Steering ratio (estimated)
N = 4000             # No. of steps [-]
dt = 0.1             # Time step [s]
res = 0.1            # Resolution of the grid [m]
dstep_max = ((np.pi / 180) / Sr) * 1000 * dt  # fminbond searches
res_rp = 0.1         # resolution of plotted road (no computations)
wh = 20              # width for heading calculations (non-critical)
L = 1.9887           # Vehicle wheel base [m]
cw_ego = 2.0         # Vehicle width [m]
dVmax = 4 * dt       # Vehicle max decel and accel
dSmax = ((np.pi / 180) / Sr) * 20 * dt  # Vehicle max steering velocity 20deg/s
tla_h = 0.5          # Heading controller: Look ahead time [s]
kh = 4.0 * 0.05      # Heading controller: Gain [-]

# === DRF (Risk field) parameters (Table 1) ===
tla = 3.5            # tla : Look ahead time [s]
par1 = 0.0064        # p : Steepness of the parabola
kexp1 = 0 * 0.07275  # k1 : inside circle
kexp2 = 19 * 0.07275  # k2 : outside circle
mcexp = 0.001        # m
cexp = 0.5           # c : ego car width/4

# === DRF driver Model parameters (Table 2) ===
Ct = 3000  # 3000(normal), 5200(sport) : Cost threshold
Vdes = 21.6  # 21.6(normal),  26(sport) : Desired speed [m/s]
kv = dt * 70e-2 / 5  # (Normal: dt * 70e-2 / 5, Sport: dt * 150e-2 / 5): acceleration gain
kvc = (1 * 0.75 * 1e-3 / 5)  # same for both : cost to velocity conversion [-]

# === Driving scene parameters (Table 3) ===
Env = 500  # C_env : surrounding environment cost
# Assuming the generate_road_parameters and obstacle_parameters functions are defined in Python
Pipe_sequence, Is = generate_road_parameters()  # Create the cost map of road
Obs_sequence, Os = obstacle_parameters()  # Create the cost map of obstacles
# Assuming the Vector_road_circuit function is defined in Python
circuit_x_ob, circuit_x_lc, circuit_x_ib, circuit_y_ob, circuit_y_lc, circuit_y_ib = Vector_road_circuit(Pipe_sequence, Is, res_rp)

# === Initializing vectors ===
Xv = np.empty(N) * np.nan      # current vehicle X position
Yv = np.empty(N) * np.nan      # current vehicle Y position
Phiv = np.empty(N) * np.nan    # current vehicle Phi angle
Delta = np.empty(N) * np.nan   # current vehicle steering angle
V = np.empty(N) * np.nan       # current vehicle speed
Lat_Dev = np.empty(N) * np.nan # current vehicle lateral deviation
Car_Nrp_Idx = np.empty(N) * np.nan
